<a href="https://colab.research.google.com/github/head1ton/llm_a_z_finetuning/blob/main/notebook/Function_Calling_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain openai tiktoken yfinance langchain_community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.9/475.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# import os

# os.environ["OPENAI_API_KEY"] = "여러분의 Key 값"

In [48]:
import os
from typing import Literal

# from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# load_dotenv()

from google.colab import userdata


def _resolve_api_context() -> tuple[str, str]:
    """선택된 API 키와 베이스 URL 정보를 반환합니다."""
    # api_key = os.getenv("OPENROUTER_API_KEY")
    api_key = userdata.get('OPENROUTER_API_KEY')

    if not api_key:
        raise RuntimeError("OPENROUTER_API_KEY가 필요합니다.")

    base_url = os.getenv("OPENROUTER_API_BASE") or "https://openrouter.ai/api/v1"

    return (api_key, base_url)


def create_openrouter_llm(
    model: str = "openai/gpt-4.1-mini",
    temperature: float = 0.3,
    max_tokens: int | None = None,
    **kwargs: object,
) -> ChatOpenAI:
    """OpenAI 호환 LLM 생성 헬퍼.

    Args:
        model: 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/gpt-4o, anthropic/claude-3-sonnet, google/gemini-pro)
        temperature: 생성 온도 (0.0-2.0)
        max_tokens: 최대 생성 토큰 수

    Returns:
        ChatOpenAI: 설정된 LLM 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    openai_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "temperature": temperature,
        "max_retries": 3,
        "timeout": 60,
        **kwargs,
    }
    if max_tokens is not None:
        openai_kwargs["max_tokens"] = max_tokens
    if base_url:
        openai_kwargs["base_url"] = base_url
    return ChatOpenAI(**openai_kwargs)

def create_embedding_model(
    model: str = "openai/text-embedding-3-small",
    **kwargs,
) -> OpenAIEmbeddings:
    """OpenAI 호환 임베딩 모델 생성.

    Args:
        model: 임베딩 모델 이름. OpenRouter에서는 provider/model 형식 사용 가능
               (예: openai/text-embedding-3-small, openai/text-embedding-3-large)
        **kwargs: 추가 파라미터 (encoding_format 등은 model_kwargs로 전달됨)

    Returns:
        OpenAIEmbeddings: 설정된 임베딩 모델 인스턴스
    """
    api_key, base_url = _resolve_api_context()

    # 전달받은 kwargs에서 model_kwargs로 전달할 파라미터 분리
    # encoding_format, extra_headers 등은 model_kwargs로 전달
    model_kwargs: dict = {}
    embedding_kwargs: dict = {
        "model": model,
        "api_key": api_key,
        "show_progress_bar": True,
        "skip_empty": True,
    }

    # 전달받은 kwargs 처리
    for key, value in kwargs.items():
        # OpenRouter API 특정 파라미터는 model_kwargs로 전달
        if key in ("encoding_format"):
            model_kwargs[key] = value
        else:
            # 나머지는 OpenAIEmbeddings에 직접 전달
            embedding_kwargs[key] = value

    if base_url:
        embedding_kwargs["base_url"] = base_url

    # model_kwargs가 있으면 전달
    if model_kwargs:
        embedding_kwargs["model_kwargs"] = model_kwargs

    return OpenAIEmbeddings(**embedding_kwargs)


def create_embedding_model_direct(
    model: str = "qwen/qwen3-embedding-0.6b",
    encoding_format: Literal["float", "base64"] = "float",
    input_text: str | list[str] = "",
    **kwargs,
) -> list[float] | list[list[float]]:
    """OpenAI SDK를 직접 사용하여 임베딩 생성 (encoding_format 지원).

    LangChain의 OpenAIEmbeddings가 encoding_format을 지원하지 않을 때 사용.

    Args:
        model: 임베딩 모델 이름
        encoding_format: 인코딩 형식 ("float")
        input_text: 임베딩할 텍스트 (문자열 또는 문자열 리스트)
        **kwargs: 추가 파라미터

    Returns:
        임베딩 벡터 리스트 (단일 텍스트) 또는 리스트의 리스트 (여러 텍스트)
    """
    from openai import OpenAI

    api_key, base_url = _resolve_api_context()

    client = OpenAI(
        base_url=base_url,
        api_key=api_key,
    )

    # input_text가 비어있으면 kwargs에서 가져오기
    if not input_text:
        input_text = kwargs.get("input", "")

    response = client.embeddings.create(
        model=model,
        input=input_text,
        encoding_format=encoding_format,
    )

    # 단일 텍스트인 경우 첫 번째 임베딩 반환
    if isinstance(input_text, str):
        return response.data[0].embedding
    else:
        # 여러 텍스트인 경우 모든 임베딩 반환
        return [item.embedding for item in response.data]


def get_available_model_types() -> dict[str, list[str]]:
    """OpenRouter에서 사용 가능한 모델 유형을 반환합니다.

    Returns:
        dict[str, list[str]]: 모델 유형별 모델 목록
    """
    return {
        "chat": [
            "openai/gpt-4.1",
            "openai/gpt-4.1-mini",
            "openai/gpt-5",
            "openai/gpt-5-mini",
            "anthropic/claude-sonnet-4.5",
            "anthropic/claude-haiku-4.5",
            "google/gemini-2.5-flash-preview-09-2025",
            "google/gemini-pro-2.5",
            "x-ai/grok-4-fast",
            "moonshotai/kimi-k2-thinking",
            "liquid/lfm-2.2-6b",
            "z-ai/glm-4.6",
        ],
        "embedding": [
            "openai/text-embedding-3-small",
            "openai/text-embedding-3-large",
            "google/gemini-embedding-001",
            "qwen/qwen3-embedding-0.6b",
            "qwen/qwen3-embedding-4b",
            "qwen/qwen3-embedding-8b",
        ],
    }


embeddings = create_embedding_model()
llm = create_openrouter_llm()

In [49]:
from langchain_core.messages import SystemMessage, HumanMessage
# from src.llm.client import create_openrouter_llm

# LLM은 한 번만 생성해서 재사용 (중요)
llm = create_openrouter_llm(
    model="openai/gpt-oss-20b:free",  # 또는 openai/gpt-4o, gpt-oss-20b 등
    temperature=0
)

## Function Calling 함수 명세 예시

Function Calling은 `LLM에게 함수에 대해서 설명`하고, 사용자의 요청 사항이 해당 함수를 통해 풀어야 하는 문제라고 판단되면 함수를 호출하는 태스크입니다.  

앞서 배웠던 ReAct 에이전트와 매우 유사하지만, ReAct 에이전트와는 달리 일반적으로는 별도의 `생각` 과정을 거치지 않고 호출한다는 점이 특징입니다.  

`LLM에게 함수에 대해서 설명하는 방식`이 정형화 된 편인데 이 부분을 유념하면 큰 어려움은 없을 것입니다.

## Function Calling 실습

### 함수 명세에 대한 간단 예시

일반적으로 Function Calling을 하기 위해서는 함수의 명세를 아래와 같은 형식으로 작성해야 합니다.  

LLM에게 아래의 함수 명세를 전달하면 LLM은 사용자가 특정 도시나 주에 대해서 온도 단위만 알면 함수를 호출할 수 있다는 것을 인지한 상태에서 사용자가 특정 지역의 온도를 물어보면 함수를 호출할 것입니다.

In [50]:
functions = [
   {
       "name": "get_current_weather",
       "description": "현재 날씨 정보 가져오기",
       "parameters": {
           "type": "object",
           "properties": {
               "location": {
                   "type": "string",
                   "description": "도시와 주/도, 예: San Francisco, CA",
               },
               "format": {
                   "type": "string",
                   "enum": ["celsius", "fahrenheit"],
                   "description": "사용할 온도 단위. 사용자의 위치에서 이를 추론합니다.",
               },
           },
           "required": ["location", "format"],
       },
   }
]

### 야후파이낸스 함수 셋팅

Function Calling 문제를 풀어보기 위해서는 호출할 수 있는 함수가 먼저 있어야 할 것입니다. 우리가 실제로 사용할 함수를 작성합니다.  
해당 함수는 미국 주식 심볼을 입력하면 해당 주식의 종가를 반환하는 함수입니다.

In [51]:
import yfinance as yf

def get_stock_price(symbol):
    """
    주어진 주식 심볼의 최신 종가를 반환하는 함수

    :param symbol: 주식 심볼 (예: 'AAPL' for Apple Inc.)
    :return: 소수점 둘째 자리까지 반올림된 최신 종가
    """
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'].iloc[0], 2)

In [52]:
print(get_stock_price('AAPL'))

278.28


In [53]:
print(get_stock_price('GOOG'))

310.52


In [54]:
print(get_stock_price('NVDA'))

175.02


### Setting up tools

함수를 작성하면 끝난 것이 아닙니다. Function Calling을 하기 위해서는 해당 함수를 특정 형식으로 작성해야 합니다.

In [55]:
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain_classic.agents import initialize_agent, Tool, AgentType
from langchain_openai import ChatOpenAI

1. 입력 데이터 형식 정의 (첫 번째 클래스)

이 부분은 함수에 어떤 데이터가.필요한지 정의합니다. 여기서는:

- StockPriceCheckInput이라는 클래스를 만듭니다.
이 클래스는 stockticker라는 문자열 데이터가 필요하다고 정의합니다 (예: "AAPL").
Field(...)는 이 데이터가 반드시 필요하다는 뜻입니다.

왜 이런 클래스가 필요할까요? AI가 이 도구를 사용할 때 "어떤 형식의 데이터를 넣어야 하는지" 알 수 있게 해줍니다. 마치 입력 양식과 같은 역할입니다.

In [56]:
from pydantic import BaseModel, Field

class StockPriceCheckInput(BaseModel):
    """Input for Stock price check."""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")

2. 도구 정의 (두 번째 클래스)


In [57]:
class StockPriceTool(BaseTool):
    name: str = "get_stock_ticker_price"
    description: str = "주식 가격을 알아야 할 때 유용합니다. yfinance API에서 사용되는 주식 티커(종목 코드)를 입력해야 합니다."

    def _run(self, stockticker: str):
        # print("i'm running")
        price_response = get_stock_price(stockticker)
        return price_response

    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput

이 부분은 실제 AI가 사용할 수 있는 도구를 만듭니다:

- StockPriceTool이라는 클래스를 만듭니다.
- name: 이 도구의 이름입니다. AI는 이 이름으로 도구를 인식합니다.
- description: 이 도구가 무엇을 하는지 설명합니다. AI는 이 설명을 읽고 언제 이 도구를 사용해야 할지 결정합니다.
- \_run: 이 도구가 실행될 때 실제로 동작하는 코드입니다. \_run은 반드시 필요하며, 이름 앞에 언더스코어(_)가 있는 건 LangChain의 규칙입니다.
- args_schema: 앞서 만든 입력 형식 클래스를 연결합니다. 이렇게 하면 AI가 이 도구에 필요한 입력 형식을 알 수 있습니다.

왜 클래스가 두 개인가요?

- 첫 번째 클래스(StockPriceCheckInput): 도구에 필요한 입력 데이터의 형식을 정의합니다.
- 두 번째 클래스(StockPriceTool): 실제 도구 자체를 정의하고, 어떻게 동작할지 구현합니다.

이렇게 분리하면 여러 도구가 같은 입력 형식을 공유하거나, 한 도구가 여러 입력 형식을 지원하는 등 더 유연하게 구성할 수 있습니다.

- _run 메서드는 LangChain의 BaseTool 클래스에서 요구하는 필수 메서드입니다.
- 이 메서드는 도구가 실제로 실행될 때 호출되는 부분으로, 도구의 핵심 기능을 구현합니다. 따라서 실제로 실행되는 부분(함수)이어야 합니다.

## 랭체인을 이용한 Function Calling

In [58]:
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ChatMessage, FunctionMessage
from langchain_classic.tools import MoveFileTool, format_tool_to_openai_function

In [59]:
# model = ChatOpenAI(model="gpt-4o")

In [60]:
model = create_openrouter_llm(
    model="openai/gpt-oss-20b:free",
    # model="openai/gpt-oss-120b:free",
    temperature=0
)

In [61]:
# StockPriceTool를 랭체인이 사용하는 도구에 등록
tools = [StockPriceTool()]

# 도구를 OpenAI에서 이해할 수 있는 형식으로 서술 등록
functions = [format_tool_to_openai_function(t) for t in tools]

In [62]:
functions

[{'name': 'get_stock_ticker_price',
  'description': '주식 가격을 알아야 할 때 유용합니다. yfinance API에서 사용되는 주식 티커(종목 코드)를 입력해야 합니다.',
  'parameters': {'properties': {'stockticker': {'description': 'Ticker symbol for stock or index',
     'type': 'string'}},
   'required': ['stockticker'],
   'type': 'object'}}]

In [63]:
# GPT-4o와 함수의 연결: Function Calling
# GPT-4o에게 위의 '함수 명세'와 '사용자의 질의'를 입력
# ai_message = model.predict_messages([HumanMessage(content="구글 주식의 가격을 알려줘")], functions=functions)
ai_message = model.invoke([HumanMessage(content="구글 주식의 가격을 알려줘")], tools=functions)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1765843200000'}, 'provider_name': None}}, 'user_id': 'user_2uZsdCzhH6Ef352baQVbUsRAeVp'}

In [ ]:
# 결과로 나오는 것은 LLM의 답변이 아니라 어떤 함수를 어떤 파라미터로 호출해야 할 것인지를 알려준다.
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"stockticker":"GOOGL"}', 'name': 'get_stock_ticker_price'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 90, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'function_call', 'logprobs': None}, id='run-a5b3a411-bce0-4a97-9939-55b838646d5d-0', usage_metadata={'input_tokens': 90, 'output_tokens': 21, 'total_tokens': 111, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

위의 ai_message에서 'function_call'이라고 적힌 부분에 주목한다.  
GPT-4o는 현재 주어진 함수 명세와 사용자의 질의를 고려하였을 때, `어떤 함수`를 `어떤 파라미터 값`으로 호출해야하는지 제안한다.

In [ ]:
ai_message.additional_kwargs['function_call']

{'arguments': '{"stockticker":"GOOGL"}', 'name': 'get_stock_ticker_price'}

GPT-4o는 주어진 함수 명세와 사용자의 질의를 고려하였을 때  

get_stock_tiker_price라는 함수를 {"stockticker":"GOOGL"}라는 파라미터로 호출해야 한다.  

따라서 이를 실제로 호출한다.

In [ ]:
# GPT-4o가 제안한 파라미터 값을 문자열로 추출
ai_message.additional_kwargs['function_call'].get('arguments')

'{"stockticker":"GOOGL"}'

In [ ]:
# GPT-4o가 제안한 파라미터 값이 현재 문자열 타입이므로 파이썬의 Dictionary 형태로 읽는다.
args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
print(args)

{'stockticker': 'GOOGL'}


`구글 주식의 가격을 알려줘`의 질문에 대한 답변은 `{'stockticker': 'GOOGL'}`인 것이다.  
이제 함수에 실제로 argument를 전달하여 함수의 결과를 얻어보자.

In [ ]:
tools

[StockPriceTool()]

In [ ]:
tools[0]

StockPriceTool()

In [ ]:
# 함수 호출
tool_result = tools[0](args)

# 함수 호출 결과를 문자열로 변환
tool_result = str(tool_result)
print(tool_result)

170.92


<ipython-input-24-d59fedc09903>:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  tool_result = tools[0](args)


In [ ]:
# 함수 호출 결과를 최종 답변에 활용하기 위해서 랭체인의 형식으로 변환
FunctionMessage(name='get_stock_ticker_price', content=tool_result)

FunctionMessage(content='170.92', additional_kwargs={}, response_metadata={}, name='get_stock_ticker_price')

In [ ]:
final_message = model.invoke([HumanMessage(content='구글 주식의 가격을 알려줘'), # 사용자의 질문
                                        ai_message, # GPT-4o가 결정한 함수와 파라미터의 값
                                        FunctionMessage(name='get_stock_ticker_price',content=tool_result)], # 함수 호출 결과
                                        tools=functions) #함수의 명세

In [ ]:
# 각종 값들이 혼재되어져 있다. 실제 답변은 .content로 접근한다.
final_message

AIMessage(content='현재 구글(알파벳) 주식의 가격은 170.92 달러입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 124, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'stop', 'logprobs': None}, id='run-0635d362-3fd4-4b17-a5fc-5c70e832150b-0', usage_metadata={'input_tokens': 124, 'output_tokens': 24, 'total_tokens': 148, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
final_message.content

'현재 구글(알파벳) 주식의 가격은 170.92 달러입니다.'

## Function Calling 에이전트

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [ ]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS, # Funtion Calling Agent
                        verbose=True)

<ipython-input-31-3bfd662c9ca4>:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  open_ai_agent = initialize_agent(tools,


In [ ]:
result = open_ai_agent.run("구글 주식의 가격은?")

<ipython-input-32-98774a12db8f>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = open_ai_agent.run("구글 주식의 가격은?")




> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


170.92현재 구글(GOOGL) 주식의 가격은 $170.92입니다.

> Finished chain.


In [ ]:
print(result)

현재 구글(GOOGL) 주식의 가격은 $170.92입니다.


## ReACT 에이전트 Vs. Function Calling 에이전트 비교

- Function Calling은 CoT(생각 과정)이 없는 도구 호출 에이전트이다.
- 이전에 ReACT 에이전트로 구현되었던 많은 프로젝트가 Function Calling으로 전환되고 있다. 이유는 CoT라는 선 과정이 토큰 소모와 느린 답변 속도에 영향을 미친다는 점때문에 모델 성능이 좋아지면서 CoT 없이도 도구 호출이 정확하게 가능한 경우가 많아졌기 때문이다.
- 하지만 반대로 Function Calling이 파인 튜닝 후에도 잘 동작하지 않는다면 성능을 올리기 위해서 다시 CoT를 추가하여 ReACT 에이전트로 변경하는 것을 고려하기도 한다. CoT는 문제를 풀기 전 생각 과정을 추가하여 성능을 올리는 방법이라는 점을 기억한다.

### 장점
- ReACT 에이전트 대비 적은 토큰 필요. 답변 시 CoT가 없기 때문이다.
- ReACT는 기본적으로 CoT를 하기 때문에 답변 속도가 느리다. CoT를 하는 동안에 시간은 계속 흐르고, CoT가 끝나야만 답변이 가능하기 때문이다. Function Calling은 더 빠른 답변을 얻을 수 있다.

### 단점
- CoT가 없기 때문에 제대로 작동하지 않을 경우 프롬프트 변경 등을 통한 커스터마이징이 쉽지 않다. Function Calling은 도구 설명 정도밖에 수정을 하지 못한다. 반면, ReACT 에이전트는 CoT라는 도구 호출 전 생각 과정이 있기때문에 해당 생각 부분에 대한 프롬프트 엔지니어링이 가능하기 때문이다.

### 오픈 모델 파인 튜닝
- 파인 튜닝이 가능한 상황이라면 특정 시나리오에 대해서 특정 함수를 호출하도록 학습하여 해당 도메인 특화 에이전트를 개발하는 것이 가능하다.  
- 또한, 파인 튜닝이 가능하다면 ReACT 에이전트보다는 CoT가 없는 Function Calling으로 성능을 먼저 보는 것을 권장하며 (토큰 절약, 인퍼런스 속도를 감안하면 Function Calling이 더 낫기 때문.) 이후 성능이 오르지 않을 경우에 CoT를 추가하여 ReACT 형식으로 개조하는 방향으로 진행하는 것을 권한다.

이 아래는 강의 영상에는 없지만 추가적으로 제공해드리는 자료입니다.

## 다수의 도구 시나리오

다수의 도구를 사용하는 Function Calling 에이전트를 만들어보자. 우선 두 개의 함수를 추가로 만든다.  

- calculate_performance: 특정 주식과 기간을 주면 해당 기간 동안의 변동률을 계산한다.  
- get_best_performing: 다수의 주식과 기간을 주면 해당 기간 동안 가장 좋은 성과를 보인 주식을 반환.


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

In [ ]:
def calculate_performance(symbol, days_ago):
    """
    주어진 기간 동안 특정 주식의 성과(가격 변동률)를 계산하는 함수.

    :param symbol: 주식 심볼 (예: 'AAPL' for Apple Inc.)
    :param days_ago: 몇 일 전부터의 성과를 계산할지 지정하는 정수
    :return: 주식의 가격 변동률 (백분율)
    """
    # 주어진 심볼에 대한 Ticker 객체 생성
    ticker = yf.Ticker(symbol)

    # 현재 날짜를 종료 날짜로 설정
    end_date = datetime.now()

    # 시작 날짜 계산 (현재로부터 days_ago일 전)
    start_date = end_date - timedelta(days=days_ago)

    # 날짜를 'YYYY-MM-DD' 형식의 문자열로 변환
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')

    # 지정된 기간의 주가 데이터 가져오기
    historical_data = ticker.history(start=start_date, end=end_date)

    # 시작 날짜의 종가와 마지막 날짜의 종가 가져오기
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]

    # 퍼센트 변화율 계산
    # ((새 가격 - 옛 가격) / 옛 가격) * 100
    percent_change = ((new_price - old_price) / old_price) * 100

    # 결과를 소수점 둘째 자리까지 반올림하여 반환
    return round(percent_change, 2)

In [ ]:
def get_best_performing(stocks, days_ago):
    """
    주어진 주식 목록에서 지정된 기간 동안 가장 좋은 성과를 보인 주식을 찾는 함수.

    :param stocks: 주식 심볼 리스트 (예: ['AAPL', 'GOOGL', 'MSFT'])
    :param days_ago: 몇 일 전부터의 성과를 계산할지 지정하는 정수
    :return: 튜플 (최고 성과 주식 심볼, 해당 주식의 성과율)
    """
    best_stock = None  # 최고 성과 주식을 저장할 변수
    best_performance = None  # 최고 성과율을 저장할 변수

    # 주어진 주식 목록을 순회
    for stock in stocks:
        try:
            # 각 주식의 성과 계산
            performance = calculate_performance(stock, days_ago)

            # 현재 주식의 성과가 지금까지의 최고 성과보다 좋으면 업데이트
            if best_performance is None or performance > best_performance:
                best_stock = stock
                best_performance = performance
        except Exception as e:
            # 성과 계산 중 오류 발생 시 오류 메시지 출력
            print(f"Could not calculate performance for {stock}: {e}")

    # 최고 성과 주식과 그 성과율 반환
    return best_stock, best_performance

get_best_performing의 함수 호출 예시를 봅시다.

In [ ]:
# 분석할 주식 목록 정의
stocks = ['AAPL', 'MSFT', 'GOOG']  # Apple, Microsoft, Google의 주식 심볼

# 분석 기간 설정 (일 단위)
days_ago = 90  # 지난 90일 동안의 성과를 분석

# get_best_performing 함수를 호출하여 최고 성과 주식과 그 성과율 얻기
best_stock, best_performance = get_best_performing(stocks, days_ago)

# 결과 출력
print(f"지난 {days_ago}일 동안 가장 좋은 성과를 보인 주식은 {best_stock}이며, 성과율은 {best_performance}%입니다.")

지난 90일 동안 가장 좋은 성과를 보인 주식은 AAPL이며, 성과율은 -1.94%입니다.


### 도구 추가

In [ ]:
from typing import List, Optional, Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

##### 주식 가격 변동률 확인을 위한 도구 추가 #####
##### 위에서 구현한 calculate_performance()를 내부적으로 사용 #####
class StockChangePercentageCheckInput(BaseModel):
    """주식 가격 변동률 확인을 위한 입력 모델"""

    stockticker: str = Field(..., description="주식 또는 지수의 티커 심볼")
    days_ago: int = Field(..., description="몇 일 전부터 확인할지 지정하는 정수")

class StockPercentageChangeTool(BaseTool):
    """주식 가격 변동률을 계산하는 도구"""

    name: str = "get_price_change_percent"
    description: str = "주식 가치의 백분율 변화를 확인해야 할 때 유용합니다. yfinance API에서 사용되는 주식 티커와 변화를 확인할 일수를 입력해야 합니다."

    def _run(self, stockticker: str, days_ago: int) -> float:
        """
        주어진 주식의 가격 변동률을 계산합니다.
        :param stockticker: 주식 티커 심볼
        :param days_ago: 몇 일 전부터 계산할지 지정하는 정수
        :return: 가격 변동률
        """
        price_change_response = calculate_performance(stockticker, days_ago)
        return price_change_response

    args_schema: Optional[Type[BaseModel]] = StockChangePercentageCheckInput
##### 주식 가격 변동률 확인을 위한 도구 추가 끝 #####

In [ ]:
##### 최고 성과 주식 찾기를 위한 입력 도구 추가 #####
##### 위에서 구현한 get_best_performing()를 내부적으로 사용 #####
class StockBestPerformingInput(BaseModel):
    """최고 성과 주식 찾기를 위한 입력 모델"""

    stocktickers: List[str] = Field(..., description="주식 또는 지수의 티커 심볼 리스트")
    days_ago: int = Field(..., description="몇 일 전부터 확인할지 지정하는 정수")

class StockGetBestPerformingTool(BaseTool):
    """여러 주식 중 최고 성과 주식을 찾는 도구"""

    name: str = "get_best_performing"
    description: str = "특정 기간 동안 여러 주식의 성과를 확인해야 할 때 유용합니다. yfinance API에서 사용되는 주식 티커 리스트와 변화를 확인할 일수를 입력해야 합니다."

    def _run(self, stocktickers: List[str], days_ago: int) -> tuple:
        """
        주어진 주식 리스트에서 최고 성과 주식을 찾습니다.
        :param stocktickers: 주식 티커 심볼 리스트
        :param days_ago: 몇 일 전부터 계산할지 지정하는 정수
        :return: 최고 성과 주식과 그 성과율
        """
        price_change_response = get_best_performing(stocktickers, days_ago)
        return price_change_response

    args_schema: Optional[Type[BaseModel]] = StockBestPerformingInput
##### 최고 성과 주식 찾기를 위한 입력 도구 추가 끝 #####

도구 3개를 모두 선택지로 제안합니다.

In [ ]:
tools = [StockPriceTool(), StockPercentageChangeTool(), StockGetBestPerformingTool()]

In [ ]:
tools

[StockPriceTool(), StockPercentageChangeTool(), StockGetBestPerformingTool()]

### 추가된 도구로부터 에이전트 호출

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [ ]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [ ]:
open_ai_agent.run("오늘 구글 주식은?")



> Entering new AgentExecutor chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


167.01오늘 구글 주식의 가격은 $167.01입니다.

> Finished chain.


'오늘 구글 주식의 가격은 $167.01입니다.'

In [ ]:
open_ai_agent.run("구글의 주가가 지난 90일 동안 상승했나요? 상승했다면 얼마나 상승했나요?")



> Entering new AgentExecutor chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


18.4구글(GOOGL)의 주가는 지난 90일 동안 18.4% 상승했습니다.

> Finished chain.


'구글(GOOGL)의 주가는 지난 90일 동안 18.4% 상승했습니다.'

In [ ]:
open_ai_agent.run("구글의 주가가 지난 3개월 동안 얼마나 상승했습니까?")



> Entering new AgentExecutor chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


-4.11구글(GOOGL)의 주가는 지난 3개월 동안 약 4.11% 하락했습니다.

> Finished chain.


'구글(GOOGL)의 주가는 지난 3개월 동안 약 4.11% 하락했습니다.'

In [ ]:
open_ai_agent.run("구글, 메타, 마이크로소프트 중 어느 주식이 지난 3개월 동안 가장 좋은 성과를 보였습니까?")



> Entering new AgentExecutor chain...

Invoking: `get_best_performing` with `{'stocktickers': ['GOOGL', 'META', 'MSFT'], 'days_ago': 90}`


('META', 6.81)지난 3개월 동안 메타(META) 주식이 6.81% 상승하며 구글과 마이크로소프트보다 더 좋은 성과를 보였습니다.

> Finished chain.


'지난 3개월 동안 메타(META) 주식이 6.81% 상승하며 구글과 마이크로소프트보다 더 좋은 성과를 보였습니다.'

In [ ]:
open_ai_agent.run("MSFT(마이크로소프트)의 주가가 지난 3개월 동안 얼마나 상승했습니까?")



> Entering new AgentExecutor chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'MSFT', 'days_ago': 90}`


-11.01마이크로소프트(MSFT)의 주가는 지난 3개월 동안 약 11.01% 하락했습니다.

> Finished chain.


'마이크로소프트(MSFT)의 주가는 지난 3개월 동안 약 11.01% 하락했습니다.'

In [ ]:
open_ai_agent.run("비트코인이 지난 3개월 동안 얼마나 상승했습니까?")



> Entering new AgentExecutor chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'BTC-USD', 'days_ago': 90}`


-12.86비트코인의 가격은 지난 3개월 동안 약 12.86% 하락했습니다.

> Finished chain.


'비트코인의 가격은 지난 3개월 동안 약 12.86% 하락했습니다.'

In [ ]:
open_ai_agent.run("비트코인이 지난 3개월 동안 얼마나 상승했는지랑 구글이 5개월 간 얼마나 상승했는지 알려줘")



> Entering new AgentExecutor chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'BTC-USD', 'days_ago': 90}`


-12.86
Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 150}`


2.59비트코인은 지난 3개월 동안 약 12.86% 하락했습니다. 반면에 구글(알파벳)은 지난 5개월 동안 약 2.59% 상승했습니다.

> Finished chain.


'비트코인은 지난 3개월 동안 약 12.86% 하락했습니다. 반면에 구글(알파벳)은 지난 5개월 동안 약 2.59% 상승했습니다.'